# Financial Porfolios 

## Excursus: Optimization with scipy

In [1]:
import pandas as pd
import numpy as np

pd.options.display.float_format = "{:.4f}".format
np.set_printoptions(suppress=True)

In [2]:
stocks = pd.read_csv("data/port_stocks.csv", parse_dates=["Date"], index_col="Date")

In [3]:
stocks  # stock prices 2014-2018

,AMZN,BA,DIS,IBM,KO,MSFT
Date,,,,,,
2013-12-31,398.7900,117.8600,71.1700,151.0700,34.8500,32.9100
2014-01-02,397.9700,118.0100,71.0500,149.4200,34.3000,32.6900
2014-01-03,396.4400,118.8300,70.9000,150.3200,34.1300,32.4700
2014-01-06,393.6300,119.5200,70.6300,149.8000,33.9700,31.7800
2014-01-07,398.0300,121.3300,71.1200,152.7900,34.0800,32.0300
...,...,...,...,...,...,...
2018-12-21,1377.4500,301.3000,104.2200,108.3700,47.1600,97.4500
2018-12-24,1343.9600,291.0200,100.3500,105.0800,45.5600,93.3800
2018-12-26,1470.9000,310.5800,105.8300,108.8100,46.5300,99.7600


In [4]:
ret = stocks.pct_change().dropna()

In [5]:
ret  # stock returns 2014-2018

,AMZN,BA,DIS,IBM,KO,MSFT
Date,,,,,,
2014-01-02,-0.0021,0.0013,-0.0017,-0.0109,-0.0158,-0.0067
2014-01-03,-0.0038,0.0069,-0.0021,0.0060,-0.0050,-0.0067
2014-01-06,-0.0071,0.0058,-0.0038,-0.0035,-0.0047,-0.0213
2014-01-07,0.0112,0.0151,0.0069,0.0200,0.0032,0.0079
2014-01-08,0.0098,0.0022,-0.0148,-0.0092,-0.0112,-0.0178
...,...,...,...,...,...,...
2018-12-21,-0.0571,-0.0272,-0.0260,-0.0185,0.0006,-0.0324
2018-12-24,-0.0243,-0.0341,-0.0371,-0.0304,-0.0339,-0.0418
2018-12-26,0.0945,0.0672,0.0546,0.0355,0.0213,0.0683


In [6]:
rf = 0.017  # risk free return

In [7]:
# calculate annualized portfolio return (based on weights)
def port_ret(weights):
    return ret.dot(weights.T).mean() * 252

In [8]:
# calculate annualized portfolio volatility (based on weights)
def port_vol(weights):
    return ret.dot(weights.T).std() * np.sqrt(252)

In [9]:
import scipy.optimize as sco  # import scipy optimize

In [10]:
# define function to be minimized (sco only supports minimize, not maximize)
# -> maximize sharpe ratio == minimize sharpe ratio * (-1)
def min_func_sharpe(weights):
    return (rf - port_ret(weights)) / port_vol(weights)  # sharpe ratio * (-1)

In [11]:
# number of assets
noa = len(ret.columns)
noa

6

In [12]:
# equal weights (starting point of optimization)
eweigths = np.full(noa, 1 / noa)
eweigths

array([0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.16666667])

In [13]:
# constraint: weights must sum up to 1 -> sum of weights - 1 = 0
cons = {"type": "eq", "fun": lambda x: np.sum(x) - 1}

In [14]:
# bounds: all weights shall be between 0 and 1 -> can be changed
bnds = tuple((0, 1) for x in range(noa))

In [15]:
# run optimization based on function to be minimized, starting with equal weights and based on respective bounds and constraints
opts = sco.minimize(
    min_func_sharpe, eweigths, method="SLSQP", bounds=bnds, constraints=cons
)

In [16]:
# output of optimization
opts

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -1.1960346770555421
       x: [ 2.596e-01  3.488e-01  0.000e+00  2.034e-16  0.000e+00
            3.916e-01]
     nit: 5
     jac: [-8.484e-02 -8.490e-02  5.424e-02  8.099e-01 -4.175e-02
           -8.554e-02]
    nfev: 35
    njev: 5

In [17]:
# getting the optimal weights
optimal_weights = opts["x"]
optimal_weights

array([0.25960831, 0.34875076, 0.        , 0.        , 0.        ,
       0.39164093])

In [18]:
# return of the optimal portfolio
port_ret(optimal_weights)

0.2558361965158479

In [19]:
# volatility of the optimal portfolio
port_vol(optimal_weights)

0.199690026633531

In [20]:
# sharpe ratio of the optimal portfolio
-min_func_sharpe(optimal_weights)

1.1960346770555421